In [1]:
import os
from ctc_timings import get_graph
import pandas as pd
import networkx as nx

In [2]:
ROOT_DATA_DIR = '/home/draga/PhD/data/cell_tracking_challenge/ST_Segmentations/'

In [3]:

ds_name = 'BF-C2DL-MuSC'
seq = 1
min_t = 0
max_t = 1375
corners = [(0, 0), (1036, 1070)]

In [4]:
sol_dir = os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_RES/'.format(seq))
os.makedirs(sol_dir, exist_ok=True)
sol_pth = os.path.join(sol_dir, 'maybe_solution.graphml')

# read coords
coords_path = os.path.join(ROOT_DATA_DIR, '{0}_{1:02}_coords.csv'.format(ds_name, seq))
coords = pd.read_csv(coords_path)

# build graph
graph, build_time = get_graph(coords, min_t, max_t, corners)

Making migration & division edges: 100%|██████████| 1375/1375 [00:27<00:00, 50.02it/s]


Build duration:  49.42817783355713


In [5]:
infinite_cost_edges = graph._g.es.select(cost_ge=1e10)
print(len(infinite_cost_edges))

0


In [6]:
m, flow = graph._to_gurobi_model()
m.optimize()

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2023-05-12
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16260 rows, 55846 columns and 212542 nonzeros
Model fingerprint: 0x524fc038
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [9e-02, 9e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 932 rows and 6 columns
Presolve time: 0.30s
Presolved: 15328 rows, 55840 columns, 210437 nonzeros

Ordering time: 0.02s

Barrier statistics:
 AA' NZ     : 1.501e+05
 Factor NZ  : 2.872e+05 (roughly 30 MB of memory)
 Factor Ops : 6.635e+06 (less than 1 second per iteration)
 Threads    : 3

Barrier performed 0 iterations in 0.47

In [7]:
# store on graph
store_time = graph.store_solution(m)

In [8]:
# save solution graph
sol_pth = os.path.join(sol_dir, 'maybe_solution.graphml')
nx_g = graph.convert_sol_igraph_to_nx()
nx.write_graphml_lxml(nx_g, sol_pth)

In [9]:
from traccuracy.loaders import load_ctc_data
from bmvc_metrics import load_sol
from traccuracy.matchers import CTCMatched
from traccuracy.metrics import CTCMetrics

gt_data = load_ctc_data(os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_GT/TRA/'.format(seq)))
sol_data = load_sol(sol_pth, os.path.join(ROOT_DATA_DIR, ds_name, '{0:02}_ST/SEG/'.format(seq)), gt_data.segmentation)

match = CTCMatched(gt_data, sol_data)
raw_ctc = CTCMetrics(match).results

Matching frames: 100%|██████████| 1376/1376 [00:59<00:00, 23.06it/s]


Matched 5388 out of 5411 ground truth nodes.
Matched 5388 out of 5419 predicted nodes.


Evaluating FN edges: 100%|██████████| 5410/5410 [00:09<00:00, 600.80it/s]


In [10]:
raw_ctc

{'AOGM': 778.5,
 'fp_nodes': 31,
 'fn_nodes': 23,
 'ns_nodes': 0,
 'fp_edges': 168,
 'fn_edges': 143,
 'ws_edges': 135,
 'TRA': 0.9874889513860988,
 'DET': 0.995176492330438}

: 

5132